In [46]:
!pip install jsonlines
import sys
import jsonlines

dname = "trivia"

# keys -> id, input, output
train_dict = {'id': [], 'input': [], 'output': []}
with jsonlines.open(f"../kilt_{dname}/{dname}-train-kilt.jsonl") as f:
    for i, line in enumerate(f.iter()):
        if i == 0: print(line.keys())
        train_dict['id'].append(line['id'])
        train_dict['input'].append(line['input'])
        train_dict['output'].append(line['output'])

dict_keys(['id', 'output', 'input'])


In [47]:
len(train_dict['id'])

61844

In [48]:
idx = 10

for _input, _output in zip(train_dict['input'][:idx], train_dict['output'][:idx]):
    print(f'input: {_input}\noutput: {_output}\n\n')

input: What denomination of British bank note depicts twenty prisoners crammed into one cell?
output: [{'answer': 'five  £'}, {'answer': '5 £'}, {'answer': '£5', 'provenance': [{'wikipedia_id': '270680', 'title': 'Banknotes of the pound sterling', 'start_paragraph_id': 30, 'start_character': 246, 'end_paragraph_id': 30, 'end_character': 248, 'bleu_score': 1.0, 'section': 'Section::::Question of legal tender.\n'}]}, {'answer': 'five £'}]


input: Under what name did Terry the dog achieve fame in a 1939 film?
output: [{'answer': 'TOTO (disambiguation)'}, {'answer': 'TOTO'}, {'answer': 'toto disambiguation'}, {'answer': 'toto'}]


input: Who connects the BBC news, ‘Crimewatch’, and ‘Antiques Roadshow’?
output: [{'answer': 'FIONA BRUCE'}, {'answer': 'Fiona Bruce', 'provenance': [{'wikipedia_id': '1139893', 'title': 'BBC News', 'start_paragraph_id': 51, 'start_character': 271, 'end_paragraph_id': 51, 'end_character': 282, 'bleu_score': 1.0, 'section': 'Section::::History.:2000s.\n'}]}, {'an

In [49]:
list(train_dict['input'])[:10]

['What denomination of British bank note depicts twenty prisoners crammed into one cell?',
 'Under what name did Terry the dog achieve fame in a 1939 film?',
 'Who connects the BBC news, ‘Crimewatch’, and ‘Antiques Roadshow’?',
 'Born in 1963, whose real name is Georgios Kyriacos Panayiotou?',
 'George Ford is a player and dad Mike Ford is the coach at which English rugby club?',
 'What surname is shared by the Formula 1 champions in 1962, 1968, and 1996?',
 'Minnie is the title character in which Puccini opera?',
 'To within two years either way, when did Rupert Bear first appear in the Daily Express?',
 'The Lech, Enns, Ipel and Nera are all tributaries of which major river?',
 'What river flows through the Welsh city of Newport?']

In [50]:
list(train_dict['output'])[:3]

[[{'answer': 'five  £'},
  {'answer': '5 £'},
  {'answer': '£5',
   'provenance': [{'wikipedia_id': '270680',
     'title': 'Banknotes of the pound sterling',
     'start_paragraph_id': 30,
     'start_character': 246,
     'end_paragraph_id': 30,
     'end_character': 248,
     'bleu_score': 1.0,
     'section': 'Section::::Question of legal tender.\n'}]},
  {'answer': 'five £'}],
 [{'answer': 'TOTO (disambiguation)'},
  {'answer': 'TOTO'},
  {'answer': 'toto disambiguation'},
  {'answer': 'toto'}],
 [{'answer': 'FIONA BRUCE'},
  {'answer': 'Fiona Bruce',
   'provenance': [{'wikipedia_id': '1139893',
     'title': 'BBC News',
     'start_paragraph_id': 51,
     'start_character': 271,
     'end_paragraph_id': 51,
     'end_character': 282,
     'bleu_score': 1.0,
     'section': 'Section::::History.:2000s.\n'}]},
  {'answer': 'Fiona Bruce',
   'provenance': [{'wikipedia_id': '23297505',
     'title': 'Crimewatch',
     'start_paragraph_id': 2,
     'start_character': 402,
     'end_pa

In [51]:
wiki_dict = {'input': [], 'wiki_id': [], 'title': []}

for _input, _output in zip(train_dict['input'], train_dict['output']):
    
    title_list = []; id_list = []
    for elem in _output:
        if 'provenance' in elem.keys():
            for _elem in elem['provenance']:
                title_list.append(_elem['title'])
                id_list.append(_elem['wikipedia_id'])
    #if len(set(title_list)) > 1: print(title_list)
    wiki_dict['input'].append(_input)
    wiki_dict['wiki_id'].append(id_list)
    wiki_dict['title'].append(title_list)

In [52]:
len(wiki_dict['input'])

61844

In [53]:
train_dict = {'input': [], 'wiki_id': [], 'title': []}
dev_dict = {'input': [], 'wiki_id': [], 'title': []}
test_dict = {'input': [], 'wiki_id': [], 'title': []}

for i, (_input, _id, _title) in enumerate(zip(wiki_dict['input'], wiki_dict['wiki_id'], wiki_dict['title'])):
    if i<50000:
        train_dict['input'].append(_input)
        train_dict['wiki_id'].append(_id)
        train_dict['title'].append(_title)
    elif i<50000+5000:
        dev_dict['input'].append(_input)
        dev_dict['wiki_id'].append(_id)
        dev_dict['title'].append(_title)
    elif i<50000+5000+10000:
        test_dict['input'].append(_input)
        test_dict['wiki_id'].append(_id)
        test_dict['title'].append(_title)
    else:
        continue


In [54]:
print(f'train: {len(train_dict["input"])}\ndev: {len(dev_dict["input"])}\ntest: {len(test_dict["input"])}')

train: 50000
dev: 5000
test: 6844


In [55]:
def dict2csv(_dict):
    print(f'Dict #: {len(_dict["input"])}')
    df = {'input': [], 'output': []}
    for _input, _title_list in zip(_dict['input'], _dict['title']):
        for _title in _title_list:
            df['input'].append(_input)
            df['output'].append(_title)
    print(f'Csv #: {len(df["input"])}')
    print(' ')
    return df
            
train_csv = dict2csv(train_dict)
dev_csv = dict2csv(dev_dict)
test_csv = dict2csv(test_dict)

Dict #: 50000
Csv #: 142031
 
Dict #: 5000
Csv #: 13075
 
Dict #: 6844
Csv #: 22176
 


In [56]:
import pickle
import pandas as pd

with open('train_total.pickle', 'wb') as f:
    pickle.dump(wiki_dict, f)
    
df = pd.DataFrame(train_csv)
df.to_csv(f'../kilt_{dname}/{dname}_toy_train.csv')
df = pd.DataFrame(dev_csv)
df.to_csv(f'../kilt_{dname}/{dname}_toy_dev.csv')
df = pd.DataFrame(test_csv)
df.to_csv(f'../kilt_{dname}/{dname}_toy_test.csv')

In [57]:
total_title = list(set(train_csv['output'] + dev_csv['output'] + test_csv['output']))
len(total_title)

36213

In [58]:
df = pd.DataFrame({'corpus': total_title})
df.to_csv(f'../kilt_{dname}/scale_1_kilt_corpus.csv')

In [59]:
def csv2pickle(_csv):
    output_tokid = [0]*len(_csv['output'])
    _pickle = {'input': list(_csv['input']), 'output': list(_csv['output']), 'output_tokid': output_tokid}
    return _pickle

train_p = csv2pickle(train_csv)
dev_p = csv2pickle(dev_csv)
test_p = csv2pickle(test_csv)

with open(f'../kilt_{dname}/{dname}_toy_train.pickle', "wb") as f:
    pickle.dump(train_p, f)
with open(f'../kilt_{dname}/{dname}_toy_dev.pickle', 'wb') as f:
    pickle.dump(dev_p, f)
with open(f'../kilt_{dname}/{dname}_toy_test.pickle', 'wb') as f:
    pickle.dump(test_p, f)